# Delineate catchments for all gauges in Germany

In this script, we use the [Global Watersheds delineator](https://github.com/mheberger/delineator) to delineate catchments for all measuring stations in `metadata.csv`.  

This script is based on the steps from the tutorial provided on the [Global Watersheds delineator](https://github.com/mheberger/delineator) Github page.

cite Global Watersheds delineator: 10.5281/zenodo.7314287 

## 0. Install delineator

It seems to be important to have the right versions of all required packages to make delineate.py work.  
So before executing this script, create a new `venv`, activate it and then run `pip install -r merit_hydro/delineator/requirements.txt`

In [1]:
from camelsp import get_metadata

import os
import requests
import zipfile
import numpy as np

/home/alexander/anaconda3/envs/delineator/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alexander/anaconda3/envs/delineator/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/alexander/anaconda3/envs/delineator/lib/python3.9/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. S

In [8]:
# create necessary directories
# folder for the downloaded data
datafolder = "../merit_hydro/data/"

# create the folder and subfolders if not exist
if not os.path.exists(datafolder):
    os.makedirs(datafolder)
    
    # folder for raster data
    os.makedirs(os.path.join(datafolder, "raster"))
    os.makedirs(os.path.join(datafolder, "raster", "accum_basins"))
    os.makedirs(os.path.join(datafolder, "raster", "flowdir_basins"))

    # folder for vector data
    os.makedirs(os.path.join(datafolder, "shp"))
    os.makedirs(os.path.join(datafolder, "shp", "merit_catchments"))
    os.makedirs(os.path.join(datafolder, "shp", "merit_rivers"))
    os.makedirs(os.path.join(datafolder, "shp", "catchments_simplified"))

## 1. Download MERIT-Hydro raster data

In [2]:
# download basin-scale MERIT-Hydro raster data (mghydro.com)
url_accum = ["https://mghydro.com/watersheds/rasters/accum_basins/accum22.tif",
             "https://mghydro.com/watersheds/rasters/accum_basins/accum23.tif",
             "https://mghydro.com/watersheds/rasters/accum_basins/accum24.tif"]

url_flow = ["https://mghydro.com/watersheds/rasters/flow_dir_basins/flowdir22.tif",
            "https://mghydro.com/watersheds/rasters/flow_dir_basins/flowdir23.tif",
            "https://mghydro.com/watersheds/rasters/flow_dir_basins/flowdir24.tif"]

# folder for the downloaded data
folder = "../merit_hydro/data/raster/"

# create the folder and subfolder if not exist
if not os.path.exists(folder):
    os.makedirs(folder)
    os.makedirs(os.path.join(folder, "accum_basins"))
    os.makedirs(os.path.join(folder, "flowdir_basins"))

for url in url_accum + url_flow:
    # download the file from the URL
    response = requests.get(url)

    if response.status_code == 200:
        # build the filename
        if url in url_accum:
            filename = os.path.join(folder, "accum_basins", os.path.basename(url))
        elif url in url_flow:
            filename = os.path.join(folder, "flowdir_basins", os.path.basename(url))
        
        # save the downloaded data to a file
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"Data downloaded and saved to {filename}")
    else:
        print(f"Failed to download data. Status code: {response.status_code}")

Data downloaded and saved to ../merit_hydro/data/raster/accum_basins/accum22.tif
Data downloaded and saved to ../merit_hydro/data/raster/accum_basins/accum23.tif
Data downloaded and saved to ../merit_hydro/data/raster/accum_basins/accum24.tif
Data downloaded and saved to ../merit_hydro/data/raster/flowdir_basins/flowdir22.tif
Data downloaded and saved to ../merit_hydro/data/raster/flowdir_basins/flowdir23.tif
Data downloaded and saved to ../merit_hydro/data/raster/flowdir_basins/flowdir24.tif


## 2. Download MERIT-Basins vector data

Vector data is stored on Google Drive, so we download the data manually from https://www.reachhydro.org/home/params/merit-basins.

In the folder pfaf_level_02 , download two sets of files:

- unit catchment shapefiles: 
    - `cat_pfaf_22_MERIT_Hydro_v07_Basins_v01.shp` -> `../data/shp/merit_catchments`  
    - `cat_pfaf_23_MERIT_Hydro_v07_Basins_v01.shp` -> `../data/shp/merit_catchments`
    - `cat_pfaf_24_MERIT_Hydro_v07_Basins_v01.shp` -> `../data/shp/merit_catchments`
- river flowline shapefiles: 
    - `riv_pfaf_22_MERIT_Hydro_v07_Basins_v01.shp` -> `../data/shp/merit_rivers`
    - `riv_pfaf_23_MERIT_Hydro_v07_Basins_v01.shp` -> `../data/shp/merit_rivers`
    - `riv_pfaf_24_MERIT_Hydro_v07_Basins_v01.shp` -> `../data/shp/merit_rivers`

## 3. Download simplified MERIT-Basins data

In [7]:
url = "https://mghydro.com/watersheds/share/catchments_simplified.zip"

# folder for the downloaded data
folder = "../merit_hydro/data/shp/catchments_simplified/"

# download the file from the URL
response = requests.get(url)

if response.status_code == 200:
    # build the filename
    filename = os.path.join(folder, os.path.basename(url))

    # save the downloaded data to a file
    with open(filename, "wb") as file:
        file.write(response.content)
    print(f"Data downloaded and saved to {filename}")
else:
    print(f"Failed to download data. Status code: {response.status_code}")

# unzip
with zipfile.ZipFile(filename, 'r') as zip:
    zip.extractall(folder)

# remove zip file
os.remove(filename)

Data downloaded and saved to ../merit_hydro/data/shp/catchments_simplified/catchments_simplified.zip


## 4. Create a CSV file with your desired watershed outlet points

We create the watershed outlet points csv from `metadata.csv`.  

Required columns and mapping to camels metadata:
- `id` -> provider_id
- `lat` -> lat
- `lng` -> lon  

Optional columns:
- `name` -> gauge_name
- `area` -> area

In [30]:
# get CAMELS metadata
meta = get_metadata()

# select columns
outlets = meta[["camels_id", "lat", "lon", "gauge_name", "area"]].copy()

# rename columns
outlets.columns = ["id", "lat", "lng", "name", "area"]

# currently, not all rows are filled (e.g. some stations do not have lat and lon), remove those stations
outlets = outlets.dropna().reset_index(drop=True)

# drop lat and lon when infinity
outlets = outlets[(outlets["lat"] != np.inf) & (outlets["lng"] != np.inf)].reset_index(drop=True)

# replace -999.0 in column area with nan
outlets["area"] = outlets["area"].replace(-999.0, np.nan)

# save as csv
outlets.to_csv('../merit_hydro/data/outlets.csv', index=False)


## 5. Update config.py

This step can be omitted once the catchment delineation is set up.

## 6. Run delineator.py to delineate watersheds

In [32]:
# create output dir if not exist
if not os.path.exists("../merit_hydro/output"):
    os.makedirs("../merit_hydro/output")

# create plots dir if not exist
if not os.path.exists("../merit_hydro/delineator/plots"):
    os.makedirs("../merit_hydro/delineator/plots")

In [33]:
!cd ../merit_hydro/delineator && python delineate.py

/home/alexander/anaconda3/envs/delineator/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/home/alexander/anaconda3/envs/delineator/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
Reading your outlets data in: /home/alexander/Github/camels/camelsp/merit_hydro/data/outlets.csv
Finding out which Level 2 megabasin(s) your points are in
Your watershed outlets are in 3 basin(s)

Beginning delineation for 427 outlet point(s) in Level 2 Basin #22.
Reading catchment geodata in /home/alexander/Github/camels/camelsp/merit_hydro/data/shp/merit_catchments/cat_pfaf_22_MERIT_Hydro_v07_Basins_v01